In [ ]:
#
# Get running config from an IOS-XR device and print out some
# interface information. Shows how to use the jxmlease library
# for when you'd rather have JSON but have to deal with XML!
#
from ncclient import manager
import jxmlease
import json

HOST = '127.0.0.1'
PORT = 8304
USER = 'cisco'
PASS = 'cisco'

def my_unknown_host_cb(host, fingerprint):
    return True

m = manager.connect(host=HOST, port=PORT, username=USER, password=PASS,
                    timeout=600,
                    allow_agent=False,
                    look_for_keys=False,
                    hostkey_verify=False,
                    unknown_host_cb=my_unknown_host_cb)

c = m.get_config(source='running')
r = jxmlease.parse(c.data_xml)

#for intf in r[u'data'][u'interface-configurations'][u'interface-configuration']:
#    if 'ipv4-network' in intf.keys():
#        print("{} has IPv4 address {}".format(
#                intf[u'interface-name'],
#                intf[u'ipv4-network'][u'addresses'][u'primary'][u'address']))
#    else:
#        print("{} has no IPv4 network config".format(intf[u'interface-name']))

print(r.emit_xml())
# print(json.dumps(r, indent=2))

#c = m.get('''<filter>
#  <netconf-state xmlns="urn:ietf:params:xml:ns:yang:ietf-netconf-monitoring">
#    <schemas/>
#  </netconf-state>
#</filter>''')
#r = jxmlease.parse(c.data_xml)

#for s in sorted(r[u'data'][u'netconf-state'][u'schemas'][u'schema']):
#    ident = s[u'identifier']
#    if 'openconfig' in ident:
#        print(ident)

In [ ]:
import jxmlease
from jxmlease import XMLDictNode
from jxmlease import XMLCDATANode


# <interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
r1 = XMLDictNode()
top = r1.add_node('interface-configurations', xml_attrs={'xmlns': 'http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg'})
print r1.emit_xml()

r2 = XMLDictNode() # '<interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg"/>')
r2['interface-configurations'] = ''
r2['interface-configurations'].xml_attrs = {'xmlns': 'http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg'}
print r2.emit_xml()

In [ ]:
top.xml_attrs

In [ ]:
import jxmlease

r = jxmlease.parse('<interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg"/>')
r

# DictToXML

In [ ]:
import dicttoxml

obj = { u'interface-configurations': u''}

xml = dicttoxml.dicttoxml(obj, attr_type=False, root=False)

print(xml)

# Using lxml

In [ ]:
from lxml import etree

IFMGR = 'http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg'

NC_NAMESPACE = 'urn:ietf:params:xml:ns:netconf:base:1.0'

NC = '{%s}' % NC_NAMESPACE

NSMAP = {
    'nc': NC_NAMESPACE
}

def serialize(d, xmlns=IFMGR):
    """Serialize a dictionary to XML"""
    assert len(d.keys()) == 1, 'Cannot encode more than one root element'
    name = d.keys()[0]
    root = etree.Element(name, xmlns=xmlns, nsmap=NSMAP)
    populate_element(root, d[name])

    return etree.tostring(root, pretty_print=True)


def populate_element(element, e):
    """Populates an etree with the given dictionary"""
    if type(e) is dict:
        # serialize child dictionary
        for k, v in e.iteritems():
            if type(v) is dict:
                child = etree.element(k)
                populate_element(child, v)
            else:
                if k[0] == '_':
                    attr = k[1:]
                    element.set(NC+attr, v)
                else:
                    leaf = etree.Element(k, nsmap=NSMAP)
                    leaf.text = unicode(v)
                    element.append(leaf)
    elif type(e) is list:
        k = element.tag
        if k[-1] == 's':
            name = k[:-1]
        else:
            name = k
        for item in e:
            child = etree.Element(name, nsmap=NSMAP)
            populate_element(child, item)
            element.append(child)
        
d1 = {
    u'interface-configurations': [
        {
            u'interface-name': 'GigabitEthernet0/0/0/0'
        }
    ]
}
d2 = {
    u'config': {
        u'interface-configurations': [
            {
                u'_operation': 'delete',
                u'interface-name': 'GigabitEthernet0/0/0/0'
            }
        ]
    }
}


print serialize(d1)
print serialize(d2)